In [1]:
import numpy as np
from queue import PriorityQueue
from math import ceil

PROBLEM_SIZE = 5
NUM_SETS = 20
SETS = [np.random.rand(PROBLEM_SIZE) < 0.1 for _ in range(NUM_SETS)]

class State:
    def __init__(self, taken, not_taken):
        self.taken = taken
        self.not_taken = not_taken

    def __lt__(self, other):
        return len(self.taken) < len(other.taken)

def covered(state):
    return np.any([SETS[i] for i in state.taken], axis=0)

def uncovered(state):
    return np.count_nonzero(np.logical_not(covered(state)))

def goal_check(state):
    return np.all(covered(state))

def h(state):
    largest_set_size = max(sum(s) for s in SETS)
    missing_size = PROBLEM_SIZE - np.count_nonzero(covered(state))
    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate

def f(state, heuristic):
    return len(state.taken) + h(state)

def a_star(heuristic=1):
    frontier = PriorityQueue()
    start_state = State(set(), set(range(NUM_SETS)))
    frontier.put((f(start_state, heuristic), start_state))
    came_from = {}
    cost_so_far = {}
    cost_so_far[start_state] = 0

    while not frontier.empty():
        _, current_state = frontier.get()

        if goal_check(current_state):
            path = []
            while current_state is not None:
                path.insert(0, current_state)
                current_state = came_from.get(current_state)

            return path

        for action in current_state.not_taken:
            new_state = State(current_state.taken | {action}, current_state.not_taken - {action})
            new_cost = cost_so_far[current_state] + 1

            if new_state not in cost_so_far or new_cost < cost_so_far[new_state]:
                cost_so_far[new_state] = new_cost
                priority = new_cost + f(new_state, heuristic)
                frontier.put((priority, new_state))
                came_from[new_state] = current_state

    return None

# Choose the heuristic (1, 2, or 3)
heuristic_choice = 1
solution = a_star(heuristic_choice)

if solution is not None:
    selected_subsets = solution[-1].taken
    total_size = len(selected_subsets)
    uncovered_elements = uncovered(solution[-1])
    print(f"Selected subsets: {selected_subsets}")
    print(f"Total size of selected subsets: {total_size}")
    print(f"Number of uncovered elements: {uncovered_elements}")
else:
    print("No solution found.")


Selected subsets: {1, 15}
Total size of selected subsets: 2
Number of uncovered elements: 0
